In [2]:
import pandas as pd
#train= pd.read_csv("/kaggle/input/roco-dataset/all_data/train/radiologytraindata.csv")
#train= pd.read_csv("/kaggle/input/roco-dataset/all_data/test/radiologytestdata.csv")
train= pd.read_csv("/kaggle/input/roco-dataset/all_data/validation/radiologyvaldata.csv")


In [8]:
train.shape

(8180, 3)

<a href="validation_ALL_R_CLIP_imgs.npy"> Download File </a>


In [4]:
import requests
from PIL import Image
import matplotlib.pyplot as plt

from transformers import CLIPProcessor, CLIPModel

model = CLIPModel.from_pretrained("flaviagiammarino/pubmed-clip-vit-base-patch32")
processor = CLIPProcessor.from_pretrained("flaviagiammarino/pubmed-clip-vit-base-patch32")

"""
url = "https://huggingface.co/flaviagiammarino/pubmed-clip-vit-base-patch32/resolve/main/scripts/input.jpeg"
image = Image.open(requests.get(url, stream=True).raw)
text = ["Chest X-Ray", "Brain MRI", "Abdominal CT Scan"]

inputs = processor(text=text, images=image, return_tensors="pt", padding=True)
probs = model(**inputs).logits_per_image.softmax(dim=1).squeeze()

plt.subplots()
plt.imshow(image)
plt.title("".join([x[0] + ": " + x[1] + "\n" for x in zip(text, [format(prob, ".4%") for prob in probs])]))
plt.axis("off")
plt.tight_layout()
plt.show()
"""

config.json:   0%|          | 0.00/4.42k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/605M [00:00<?, ?B/s]

/opt/conda/lib/python3.10/site-packages/torch/_utils.py:831: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()
2024-07-12 06:28:18.809065: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-07-12 06:28:18.809181: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-07-12 06:28:18.961376: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one h

preprocessor_config.json:   0%|          | 0.00/316 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/568 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/862k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/525k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.22M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/389 [00:00<?, ?B/s]

'\nurl = "https://huggingface.co/flaviagiammarino/pubmed-clip-vit-base-patch32/resolve/main/scripts/input.jpeg"\nimage = Image.open(requests.get(url, stream=True).raw)\ntext = ["Chest X-Ray", "Brain MRI", "Abdominal CT Scan"]\n\ninputs = processor(text=text, images=image, return_tensors="pt", padding=True)\nprobs = model(**inputs).logits_per_image.softmax(dim=1).squeeze()\n\nplt.subplots()\nplt.imshow(image)\nplt.title("".join([x[0] + ": " + x[1] + "\n" for x in zip(text, [format(prob, ".4%") for prob in probs])]))\nplt.axis("off")\nplt.tight_layout()\nplt.show()\n'

In [5]:
from tqdm.notebook import tqdm
import cv2

In [6]:
import numpy as np


for i in tqdm(train.name):
    try:
        a=Image.open("/kaggle/input/roco-dataset/all_data/validation/radiology/images/"+i)
        a= np.array(a.resize((480, 480),Image.ANTIALIAS))
        try:
            inputs = processor(images=a, return_tensors="pt")
            image_features = model.get_image_features(**inputs)
        except:
            inputs = processor(images=cv2.cvtColor(a, cv2.COLOR_GRAY2RGB), return_tensors="pt")
            image_features = model.get_image_features(**inputs)
        np.save("R_CLIP_imgs_"+i+".npy",image_features.detach().numpy())
    except:
        pass
     
flag= True
errores_2=[]
for i in train.name:
    try:
        a= np.load("R_CLIP_imgs_"+i+".npy",allow_pickle=True)
        if flag:
            flag=False
            matrix= a
        else:
            matrix= np.concatenate((matrix, a), axis = 0)
    except :
        errores_2.append(i)
        pass

        #break
np.save("val_ALL_R_CLIP_imgs.npy",matrix)

  0%|          | 0/8180 [00:00<?, ?it/s]

/tmp/ipykernel_33/3523907924.py:7: DeprecationWarning: ANTIALIAS is deprecated and will be removed in Pillow 10 (2023-07-01). Use LANCZOS or Resampling.LANCZOS instead.
  a= np.array(a.resize((480, 480),Image.ANTIALIAS))


In [10]:
errores_2

['PMC2946122_yjbm_83_3_113_g02.jpg',
 'PMC3277920_PHLEB-10-100-g2.jpg',
 'PMC5754747_f1000research-6-14507-g0001.jpg',
 'PMC4887302_12471_2016_832_Fig2_HTML.jpg',
 'PMC4467246_icrp2_43_f2.jpg']

In [9]:
np.save("validation_ALL_R_CLIP_imgs.npy",matrix)

In [7]:
matrix.shape

(8175, 512)

In [36]:
len(errores_2)

3

In [23]:
matrix.shape

(8176, 512)

In [12]:
errors=[]
for c,i in enumerate(train.name):
    if i in errores_2:
        errors.append(c)
        
print(errors)

[1015, 2451, 2899, 5055, 5819]


train : [6188, 6348, 9369, 13475, 17862, 19678, 19938, 20339, 21050, 21204, 21411, 21731, 23560, 27405, 28359, 29971, 30386, 40425, 40560, 42465, 44088, 46072, 48325, 50466, 50719, 53736, 56448, 60110, 61449, 62136, 63143]

TEST: [4354, 5235, 6471]

In [13]:
errores= ['PMC3999928_f1000research-3-4060-g0002.jpg',
 'PMC2892763_yjbm_83_2_73_g01.jpg',
 'PMC4445437_yjbm_88_2_157_g06.jpg',
 'PMC5754747_f1000research-6-14507-g0003.jpg',
 'PMC3999928_f1000research-3-4060-g0001.jpg',
 'PMC3999928_f1000research-3-4060-g0000.jpg',
 'PMC5407183_ofw26702.jpg',
 'PMC5569400_05-i004a.jpg',
 'PMC5241049_nihms839240f3.jpg',
 'PMC4240561_MA-68-291-g002.jpg',
 'PMC3941458_yjbm_87_1_3_g01.jpg',
 'PMC3999928_f1000research-3-4060-g0003.jpg',
 'PMC4345544_yjbm_88_1_93_g05.jpg',
 'PMC5529052_nihms879007f1.jpg',
 'PMC2946122_yjbm_83_3_113_g01.jpg',
 'PMC4603610_amjcaserep-16-715-g003.jpg',
 'PMC4156025_f1000research-3-3454-g0000.jpg',
 'PMC4603610_amjcaserep-16-715-g002.jpg',
 'PMC5628629_RCR2-5-na-g002.jpg',
 'PMC2890366_zdc0071083540001.jpg',
 'PMC5569400_05-i004b.jpg',
 'PMC4043289_nihms586074f2.jpg',
 'PMC3483062_noph36-149-f1.jpg',
 'PMC5754747_f1000research-6-14507-g0000.jpg',
 'PMC5218828_nihms839236f3.jpg',
 'PMC3892916_f1000research-2-1897-g0002.jpg',
 'PMC4345544_yjbm_88_1_93_g03.jpg',
 'PMC4345544_yjbm_88_1_93_g04.jpg',
 'PMC4728737_emss-66793-f0004.jpg',
 'PMC4887302_12471_2016_832_Fig1_HTML.jpg']

FileNotFoundError: [Errno 2] No such file or directory: 'A/ALL_R_CLIP_imgs.npy'

In [38]:
matrix.shape

(65419, 512)

In [13]:
from transformers import AutoTokenizer, CLIPModel
model = CLIPModel.from_pretrained("flaviagiammarino/pubmed-clip-vit-base-patch32")
tokenizer = AutoTokenizer.from_pretrained("flaviagiammarino/pubmed-clip-vit-base-patch32")


In [14]:
import numpy as np
textos= list(train.caption.values)
#c=0

for c in range(int(len(textos)/100)):
    inputs = tokenizer(textos[c*100:(c+1)*100], padding=True, return_tensors="pt", add_special_tokens = True,truncation=True, max_length=77)
    text_features = model.get_text_features(**inputs)
    #last_hidden_state = text_features.last_hidden_state
    #pooled_output = text_features.pooler_output
    np.save("v_ALL_R_CLIP_text_"+str(c)+".npy",text_features.detach().numpy())
    
flag= True
for c in range(int(len(textos)/100)):
    a= np.load("v_ALL_R_CLIP_text_"+str(c)+".npy",allow_pickle=True)
    if flag:
        flag=False
        matrix= a
    else:
        matrix= np.concatenate((matrix, a), axis = 0)
np.save("val_ALL_R_CLIP_text.npy",matrix)

In [16]:
matrix.shape

(8100, 512)

In [44]:
textos= list(train.caption.values)

In [17]:
len(textos)

8180

In [48]:
text_features.shape

torch.Size([100, 512])

In [20]:
#-79
inputs = tokenizer(textos[-80:], padding=True, return_tensors="pt", add_special_tokens = True,truncation=True, max_length=77)
text_features = model.get_text_features(**inputs)

np.save("last_validation_R_CLIP_text_.npy",text_features.detach().numpy())

In [19]:
np.save("validation_ALL_R_CLIP_text.npy",matrix)

<a href="last_validation_R_CLIP_text_.npy"> Download File </a>


In [40]:
len(textos)
int(len(textos)/100)

81

<a href="TEST_ALL_R_CLIP_text.npy"> Download File </a>

In [12]:
matrix.shape

(65400, 512)